In [24]:
import numpy as np
import cv2
import time
import csv

MIN_MATCH_COUNT = 19  #мінімальна кількість спільних ознак, яка потрібна для розпізнавання об'єкта
QUERY_IMAGE = "img/main.jpg" #Об'єкт, який потрібно розпізнати

In [25]:
#Код, який рахує метрики

viewImages = False
writeCSV = True
CSVname = "csv/SURF_Egor_dataresult.csv"
imgWithObj = 102
imgWithoutObj = 18

ametric1 = 0
ametric2 = 0
ametric3 = 0
ndetected = 0
print("image\t1 metric\t2 metric\tsize\t\t time\t\tgood matches")
csv_columns = ['Matches', 'MeanDist', 'Size', 'Time']
results = []

trueDetected = 0
falseDetected = 0

img1 = cv2.imread(QUERY_IMAGE, cv2.IMREAD_GRAYSCALE)
#img1 = cv2.resize(img1, (0,0), fx=0.1, fy=0.1) 

surf = cv2.xfeatures2d.SURF_create()
kp1, des1 = surf.detectAndCompute(img1, None)

for i in range(1, imgWithObj + imgWithoutObj+1):
    img2 = cv2.imread(f"img/{i}.jpg", cv2.IMREAD_GRAYSCALE)
    #if(i == 98):
        #img2 = cv2.resize(img2, (0,0), fx=0.9, fy=0.9)
    
    start_time = time.time()
    kp2, des2 = surf.detectAndCompute(img2, None)
    bf = cv2.BFMatcher()
    
    good = []
    #good = bf.match(des1, des2)  
    #good = sorted(matches, key = lambda x:x.distance)
    if isinstance(des2, np.ndarray):
        matches = bf.knnMatch(des1, des2, k=2)
        for m,n in matches:
            if m.distance < 0.75*n.distance:
                good.append(m)
                
                
    Time = round(time.time() - start_time, 4) #3 метрика: секунд
    Size = img2.shape[0] * img2.shape[1]        # пікселів
    ametric3 += Time
    
    if (len(good)>=MIN_MATCH_COUNT):
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()
        
        h,w = img1.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,M)

        img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
        
        corect = 0
        sum = 0
        for j in range(len(mask)):
            if mask[j][0] == 1:
                corect+=1
                sum += good[j].distance
        metric1 = round(corect / len(mask), 4)#1 метрика: відносна кількість правильно суміщених ознак
        metric2 = round(sum / corect, 4)#2 метрика: похибка локалізації, рахую як середнє значення distance об'єкту DMatch,
                                            #в тих випадках, коли ознака суміщена правильно.
        print(f"{i}:\t{metric1}\t\t{metric2}\t\t{Size}\t\t{Time}\t\t{len(good)}")
        ametric1 += metric1
        ametric2 += metric2
        
        
        ndetected += 1
        if(i <= imgWithObj):
            trueDetected += 1
        if(i > imgWithObj):
            falseDetected += 1
    else:
        print (f"{i}:\tObject has not detected. Not enough matches are found - {len(good)}/{MIN_MATCH_COUNT}")
        matchesMask = None
        metric1 = None
        metric2 = None
    #metric3 = round((time.time() - start_time)*10**6/(img2.shape[0] * img2.shape[1]),3)#3 метрика: секунд на 1 мегапіксель
    
    if(viewImages):
        draw_params = dict(matchColor = (0,255,0), singlePointColor = None, matchesMask = matchesMask, flags = 2)
        img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)

        cv2.imshow("Matching result", img3)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    data = {'Matches': metric1, 'MeanDist': metric2, 'Size': Size, 'Time': Time}

    results.append(data)
print(f"Average:\n1 metric: {ametric1/ndetected}\n2 metric: {ametric2/ndetected}\n3 metric: {ametric3/ndetected}")
print(f"true detection: {trueDetected/imgWithObj*100}%")
print(f"true no detection: {((imgWithoutObj-falseDetected)*100)/imgWithoutObj}%")

if(writeCSV):
    try:
        with open(CSVname, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns, lineterminator = '\n')
            writer.writeheader()
            writer.writerows(results)
    except IOError:
        print("I/O error")

image	1 metric	2 metric	size		 time		good matches
1:	0.7021		0.2128		129792		0.049		47
2:	0.6897		0.2109		129792		0.056		29
3:	0.7358		0.1877		129792		0.051		53
4:	0.5405		0.1727		129792		0.0455		37
5:	0.625		0.2168		129792		0.049		40
6:	0.3182		0.2443		129792		0.0365		22
7:	0.6452		0.2072		519168		0.1491		31
8:	0.7576		0.1993		519168		0.1486		33
9:	0.7143		0.1661		519168		0.1531		49
10:	0.6341		0.1953		519168		0.1131		41
11:	0.7297		0.1748		519168		0.1106		74
12:	0.7581		0.1782		519168		0.1171		62
13:	0.44		0.2104		519168		0.1926		25
14:	0.5909		0.207		519168		0.1221		44
15:	0.8372		0.1771		519168		0.2492		43
16:	0.6923		0.2157		519168		0.1291		26
17:	Object has not detected. Not enough matches are found - 16/19
18:	0.7372		0.1616		129792		0.0385		156
19:	0.3333		0.1956		129792		0.039		39
20:	0.2903		0.1882		129792		0.0345		31
21:	0.3333		0.2281		129792		0.0365		21
22:	0.5429		0.1819		129792		0.043		35
23:	0.5		0.2089		129792		0.0435		30
24:	0.6111		0.1951		129792		0.04		54
25:	0.6739

In [20]:
#Код для обробки конкретного зображення

Image = "img2/114.jpg"

img1 = cv2.imread(QUERY_IMAGE, cv2.IMREAD_GRAYSCALE)
img1 = cv2.resize(img1, (0,0), fx=0.1, fy=0.1) 

surf = cv2.xfeatures2d.SURF_create()
kp1, des1 = surf.detectAndCompute(img1, None)

img2 = cv2.imread(Image, cv2.IMREAD_GRAYSCALE)
img2 = cv2.resize(img2, (0,0), fx=0.2, fy=0.2)
kp2, des2 = surf.detectAndCompute(img2, None)
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)
good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append(m)
"""        
sum = 0
for m in good:
    sum += m.distance
print(f"{sum / len(good)}")
"""
if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()

    h,w = img1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)

    img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)

else:
    print ("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
    matchesMask = None


draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)

img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)

#plt.imshow(img3, 'gray'),plt.show()
    
#matching_result = cv2.drawMatches(img1, kp1, img2, kp2, matches[:20], None, flags=2)
#img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imshow("Matching result", img3)
cv2.waitKey(0)
cv2.destroyAllWindows()

Not enough matches are found - 10/550


In [7]:
for i in range(2):
    img = cv2.imread(f"{i+62}.jpg")
    #if i > 102 or i > 5 and i < 11:
    img = cv2.resize(img, (0,0), fx=0.2, fy=0.2)
    #else:
        #img = cv2.resize(img, (0,0), fx=0.1, fy=0.1)
    cv2.imwrite(f"{i+62}.jpg", img)